<a href="https://colab.research.google.com/github/julrods/cyber-bullying-detector/blob/main/2_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REFERENCIA: https://swatimeena989.medium.com/bert-text-classification-using-keras-903671e0207d#011a

- Javi: preguntar GCP
- Probar binaria de nuevo, debería ir mejor
- Este modelo es diferente al del ejemplo porque hay una Red densa metida en TFBErt for seq classification, no la ponemos manual. Por eso también le pasamos la máscara y el validation data de manera diferente al hacer fit


Next steps:
- 0. Conectar Jupyter a VM de GCP. 
- 0. Hacer predict con train para ver si hay overfitting
- 1. Reentrenar con los mismo datos (aggression): Ver vector de losses (guardar) y añadir al nuevo vector cuando reentrene. 
- 2. Hacer epochs hasta que converja (llegue al mínimo). El modelo tiene un vector de losses y otro de accuracy. Buscar el mínimo de validation. 
- 3. Probar con todos los datos: usar los pesos que tengo y volverlo a entrenar con nuevos datos (cargar pesos antes de entrenar).
- (4? Según cómo lo vea hacer resampling. )
- 5. Productizarlo: Coger datos reales y clasificarlos. Buscar usuario de twitter con tweets ofensivos + usuario no ofensivo. Spider para scrappear IG. Dataset del https://projecte-equal.com/#dades

Dudas: 
- Mirar el tema de la transformación al castellano
- Pasa algo por haber cogido el código de la chica?

VMs: 
- https://towardsdatascience.com/running-jupyter-notebook-in-google-cloud-platform-in-15-min-61e16da34d52
- https://medium.com/swlh/how-to-use-google-cloud-and-gpu-build-simple-deep-learning-environment-c6eadff2a569

# Environment

## Libraries

In [ ]:
!pip install transformers

In [2]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.utils import shuffle
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Functions

In [5]:
def load_vectors(dataset_name):
  pickle_inp_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/data/3_tokenized_data/bert_inp_{dataset_name}.pkl'
  pickle_mask_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/data/3_tokenized_data/bert_mask_{dataset_name}.pkl'
  pickle_label_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/data/3_tokenized_data/bert_label_{dataset_name}.pkl'

  input_ids = pickle.load(open(pickle_inp_path, 'rb'))
  attention_masks = pickle.load(open(pickle_mask_path, 'rb'))
  labels = pickle.load(open(pickle_label_path, 'rb'))

  return input_ids, attention_masks, labels

In [6]:
def train_test_val(input_ids, attention_masks, labels):
  train_inp, test_inp, train_mask, test_mask, train_label, test_label = train_test_split(input_ids,
                                                                                         attention_masks,
                                                                                         labels,
                                                                                         test_size = 0.2,
                                                                                         random_state = 42,
                                                                                         stratify = labels)
  
  train_inp, val_inp, train_mask, val_mask, train_label, val_label = train_test_split(train_inp,
                                                                                      train_mask,
                                                                                      train_label,
                                                                                      test_size = 0.2,
                                                                                      random_state = 42,
                                                                                      stratify = train_label)
  
  return train_inp, val_inp, test_inp, train_mask, val_mask, test_mask, train_label, val_label, test_label

In [7]:
def bert_setup():
  base_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,
                                       epsilon=1e-08)
  
  base_model.compile(loss = loss, optimizer = optimizer, metrics = [metric])
  
  return base_model

In [8]:
def train_model(base_model, new_model_name, old_model_name=None, batch_size=32, epochs=4):

  new_model = base_model
  new_model_save_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/models/{new_model_name}.h5'
  print(f'\nNew model will be saved at {new_model_save_path}')

  callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = new_model_save_path,
                                                  save_weights_only = True,
                                                  monitor = 'val_loss',
                                                  mode = 'min',
                                                  save_best_only = True)]

  if old_model_name: 
    old_model_save_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/models/{old_model_name}.h5'
    new_model.load_weights(old_model_save_path)
    print(f'\nTraining on previous model {old_model_name}')
  else:
    print('\nTraining new model')
  
  print('\nTraining starting...\n\n')
  history = new_model.fit(x = [train_inp, train_mask],
                           y = train_label,
                           batch_size = batch_size,
                           epochs = epochs,
                           validation_data = ([val_inp, val_mask], val_label), 
                           callbacks = callbacks)
  return new_model

In [9]:
def evaluate_model(model_name, inputs, mask, labels, base_model=None):
  if type(model_name) == str:
    model_save_path = f'/content/gdrive/MyDrive/Cyber-bullying-project/models/{model_name}.h5'
    trained_model = base_model.load_weights(model_save_path)
  
  else:
    trained_model = model_name
  
  preds = trained_model.predict([inputs, mask],
                                batch_size=32)
  
  pred_labels = [np.argmax(pred) for pred in preds[0]]
  f1 = f1_score(labels, pred_labels)
  class_report = classification_report(labels, pred_labels)
  conf_matrix = confusion_matrix(labels, pred_labels)

  return preds, pred_labels, f1, class_report, conf_matrix

In [10]:
def print_eval(f1, class_report, conf_matrix):
  print('F1 score:', f1)
  print('Classification Report:')
  print(class_report)
  print('Confusion Matrix:')
  print(conf_matrix)

# BERT model setup

Read about callbacks: https://keras.io/api/callbacks/

In [ ]:
base_model = bert_setup()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Fine-tuning the pre-trained BERT model

## Aggression dataset

### Loading the data from the pickle files

In [11]:
input_ids, attention_masks, labels = load_vectors('aggression')

### Spitting into train, test and validation set

In [12]:
train_inp, val_inp, test_inp, train_mask, val_mask, test_mask, train_label, val_label, test_label = train_test_val(input_ids, attention_masks, labels)

### Training the model

In [ ]:
aggression_model = train_model(base_model=base_model, new_model_name='aggression_model', old_model_name='bert_model')


New model will be saved at /content/gdrive/MyDrive/Cyber-bullying-project/models/aggression_model.h5

Training on previous model bert_model

Training starting...


Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the

### Evaluating the model

#### With train data

In [ ]:
preds_train, pred_labels_train, f1_train, class_report_train, conf_matrix_train = evaluate_model(aggression_model, train_inp, train_mask, train_label)

In [ ]:
print_eval(f1_train, class_report_train, conf_matrix_train)

F1 score: 0.9821220776473077
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     64691
           1       0.98      0.98      0.98      9461

    accuracy                           1.00     74152
   macro avg       0.99      0.99      0.99     74152
weighted avg       1.00      1.00      1.00     74152

Confusion Matrix:
[[64530   161]
 [  177  9284]]


#### With test data

In [ ]:
preds_test, pred_labels_test, f1_test, class_report_test, conf_matrix_test = evaluate_model(aggression_model, test_inp, test_mask, test_label)

In [ ]:
print_eval(f1_test, class_report_test, conf_matrix_test)

F1 score: 0.759124087591241
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     20217
           1       0.82      0.70      0.76      2956

    accuracy                           0.94     23173
   macro avg       0.89      0.84      0.86     23173
weighted avg       0.94      0.94      0.94     23173

Confusion Matrix:
[[19773   444]
 [  876  2080]]


In [ ]:
# hacer gráfico 

## Aggression dataset - training from scratch

### Training the model

In [14]:
aggression_model_new = train_model(base_model=base_model, new_model_name='aggression_model_new')


New model will be saved at /content/gdrive/MyDrive/Cyber-bullying-project/models/aggression_model_new.h5

Training new model

Training starting...


Epoch 1/4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with 

### Evaluating the model

#### With train data

In [15]:
preds_train, pred_labels_train, f1_train, class_report_train, conf_matrix_train = evaluate_model(aggression_model_new, train_inp, train_mask, train_label)

In [16]:
print_eval(f1_train, class_report_train, conf_matrix_train)

F1 score: 0.980590629239278
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     64691
           1       0.97      0.99      0.98      9461

    accuracy                           0.99     74152
   macro avg       0.98      0.99      0.99     74152
weighted avg       1.00      0.99      1.00     74152

Confusion Matrix:
[[64383   308]
 [   64  9397]]


#### With test data

In [17]:
preds_test, pred_labels_test, f1_test, class_report_test, conf_matrix_test = evaluate_model(aggression_model_new, test_inp, test_mask, test_label)

In [18]:
print_eval(f1_test, class_report_test, conf_matrix_test)

F1 score: 0.7467543138866065
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96     20217
           1       0.73      0.77      0.75      2956

    accuracy                           0.93     23173
   macro avg       0.85      0.86      0.85     23173
weighted avg       0.94      0.93      0.93     23173

Confusion Matrix:
[[19360   857]
 [  684  2272]]


In [19]:
# hacer gráfico de cuando converge y quedarme con esas epochs